<h4> Performance Metrics </h4>

In [1]:
import numpy as np
import pandas as pd

from get_labels import get_labels
from sklearn.externals import joblib
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import ShuffleSplit

import keras.backend as K
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation, Reshape
from keras.layers import LSTM
import subprocess;
import sys;

Using TensorFlow backend.


In [2]:
labels = get_labels();

/home/ubuntu/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
import os;
os.system("taskset -p 0xff %d" % os.getpid())
#import affinity
#import multiprocessing
#affinity.set_process_affinity_mask(0,2**multiprocessing.cpu_count()-1)

0

In [6]:
#Our custom loss function
def multiclass_loss(y_true, y_pred):
    EPS = 1e-5
    y_pred = K.clip(y_pred, EPS, 1 - EPS)
    return -K.mean((1 - y_true) * K.log(1 - y_pred) + y_true * K.log(y_pred))

In [9]:
train_x = joblib.load("/mnt/cleaned_tfidf_reduced_420_morning");

In [10]:
print('train_x');
train_x_reshaped = train_x.reshape(1851243,1,1000);
print('train_x_reshaped');
labels_array = np.array([x for x in labels]);

labels_reshaped = labels_array.reshape(1851243, 1, 1070);
print('labels_reshaped');
x_train, x_test, y_train, y_test = train_test_split(train_x_reshaped, labels_reshaped, test_size=0.20, random_state=1024)

print('train_test_split');
model = load_model('khot_LSTM_0904.h5', custom_objects={"multiclass_loss":multiclass_loss})
print('model');

train_x
train_x_reshaped
labels_reshaped
train_test_split
model


In [11]:
del train_x_reshaped
del labels_array
del labels_reshaped
del train_x
del x_train
del y_train

In [12]:
#predictions = np.array([89, 702, 307, 590,  93, 851, 782, 305, 349, 878, 913,  84, 663, 924, 403]);
#predictions = np.array([702, 782,  89, 305, 307, 851, 798,  93, 663,  84, 878, 590,  95, 499, 349]);

def get_preds_array():
    #all_preds = np.array((1,len(x_test)));
    #all_test = np.array((1,len(x_test)));
    all_preds = [];
    all_test = [];
    
    #k = 4;
    
    for idx, test_val in enumerate(x_test):
        y_test_val = y_test[idx];
        
        k = len(y_test_val[y_test_val == 1]);
        
        pred_val = model.predict(test_val.reshape(1,1,1000))[0][0]
        topk = pred_val.argsort()[-1 * k:][::-1]

        pred_arr = np.zeros(y_test_val.shape);
        pred_arr[0,topk] = 1;
        
        
        all_preds.extend(pred_arr[0]);
        all_test.extend(y_test_val[0]);
        
        if idx % 500 == 0:
            sys.stdout.write('\rOn ' + str(idx) + ' / ' + str(len(x_test)));
            
    return (all_preds, all_test);

(all_preds, all_test) = get_preds_array();

On 370000 / 370249

In [15]:
#joblib.dump((all_preds, all_test), 'predictions_test.pkl')
preds_arr = np.array(all_preds);

In [16]:
del all_preds
test_arr = np.array(all_test);

In [17]:
del all_test

In [21]:
print(np.sum(preds_arr));
print(np.sum(test_arr));

4645257.0
4645257.0


In [22]:
def tp_rate(predictions, actuals, get_rate=True):
    sums = predictions + actuals;
    all2s = sums[sums == 2];
    if get_rate:
        return len(all2s) / float(len(sums));
    else:
        return len(all2s);

In [23]:
def fp_rate(predictions, actuals, get_rate=True):
    act_not = np.logical_not(actuals).astype(int);
    return tp_rate(predictions, act_not, get_rate);

In [24]:
def fn_rate(predictions, actuals, get_rate=True):
    pred_not = np.logical_not(predictions).astype(int);
    return tp_rate(pred_not, actuals, get_rate);

In [25]:
def tn_rate(predictions, actuals, get_rate=True):
    pred_not = np.logical_not(predictions).astype(int);
    act_not = np.logical_not(actuals).astype(int);
    return tp_rate(pred_not, act_not, get_rate);

In [26]:
def accuracy(predictions, actuals):
    tp_val = tp_rate(predictions, actuals, False);
    tn_val = tn_rate(predictions, actuals, False);
    
    return (tp_val + tn_val) / float(len(predictions));

In [27]:
def precision(predictions, actuals, get_rate=True):
    tp = tp_rate(predictions, actuals, get_rate);
    fp = fp_rate(predictions, actuals, get_rate);
    return (float(tp) / (tp + fp));

In [28]:
def recall(predictions, actuals, get_rate=True):
    tp = tp_rate(predictions, actuals, get_rate);
    fn = fn_rate(predictions, actuals, get_rate);
    
    return (tp / float(tp + fn));

In [29]:
def confusion_array(predictions, actuals, get_rate=True):
    fp = fp_rate(predictions, actuals, get_rate);
    tp = tp_rate(predictions, actuals, get_rate);
    fn = fn_rate(predictions, actuals, get_rate);
    tn = tn_rate(predictions, actuals, get_rate);
    
    conf = np.array([[tp, fp], [fn, tn]]);

    conf_pdf = pd.DataFrame(conf);
    conf_pdf.columns = ['1', '0'];
    conf_pdf.set_index(np.array(['1', '0']))
    
    return conf_pdf;

k = variable (904 model)

In [30]:
conf_arr = confusion_array(preds_arr, test_arr)

In [31]:
conf_arr

,1,0
0,0.003037,0.008688
1,0.008688,0.979586


In [32]:
conf_vals = confusion_array(preds_arr, test_arr, False)

In [33]:
conf_vals

,1,0
0,1203247,3442010
1,3442010,388079163


In [34]:
precision_val = precision(preds_arr, test_arr);

In [35]:
precision_val

0.2590270032422318

In [36]:
recall_val = recall(preds_arr, test_arr);

In [37]:
recall_val

0.2590270032422318

In [38]:
acc_val = accuracy(preds_arr, test_arr);

In [39]:
acc_val

0.9826234141040169

In [40]:
from sklearn.metrics import roc_auc_score;

In [41]:
roc_auc_score(test_arr, preds_arr)

0.62511781342164274